In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 807.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9be12ea60246fb62986786147d4ad7a43a05b2642da0a5004b5449a96c140436
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%load_ext autoreload
%autoreload 2

Mounted at /content/drive


In [ ]:
# Load JSON dataset
output_json = "/content/drive/MyDrive/umich 25 WN/CSE 692 Project/BART/DATASET/5class_Land_larger_dis.json"


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import Dataset, DatasetDict
import json

import random

# Load pretrained BART model and tokenizer
model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Add "PFAS" as a special token
special_tokens = ["PFAS"]
tokenizer.add_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))


with open(output_json, "r", encoding="utf-8") as f:
    dataset = json.load(f)



# Shuffle for randomness
random.shuffle(dataset)

# Track already used original sentences
used_originals = set()

# Store selected examples by category
discharger_data = []
landcover_data = []
label_data = []



# How many samples per category to target
target_per_category = len(dataset) // 3
extra = len(dataset) - target_per_category * 3

for item in dataset:
    orig = item["original"]

    if orig in used_originals:
        continue

    if "masked_discharger" in item and len(discharger_data) < target_per_category:
        discharger_data.append({"input_text": item["masked_discharger"], "target_text": orig})
        used_originals.add(orig)
    elif "masked_landcover" in item and len(landcover_data) < target_per_category:
        landcover_data.append({"input_text": item["masked_landcover"], "target_text": orig})
        used_originals.add(orig)
    elif "masked_label" in item and len(label_data) < target_per_category:
        label_data.append({"input_text": item["masked_label"], "target_text": orig})
        used_originals.add(orig)

    # Once we reach the total, stop
    if len(discharger_data) + len(landcover_data) + len(label_data) >= 407:
        break

# If there's still room (e.g., 200*3 = 600, need 2 more), fill from any category
remaining = len(dataset) - (len(discharger_data) + len(landcover_data) + len(label_data))
if remaining > 0:
    remaining_data = []
    for item in dataset:
        orig = item["original"]
        if orig in used_originals:
            continue
        if "masked_label" in item:
            remaining_data.append({"input_text": item["masked_label"], "target_text": orig})
        elif "masked_discharger" in item:
            remaining_data.append({"input_text": item["masked_discharger"], "target_text": orig})
        elif "masked_landcover" in item:
            remaining_data.append({"input_text": item["masked_landcover"], "target_text": orig})
        if len(remaining_data) >= remaining:
            break

# Final dataset
hf_data = discharger_data + landcover_data + label_data + remaining_data
random.shuffle(hf_data)  # shuffle for randomness


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:
# Convert to Hugging Face Dataset format
hf_dataset = Dataset.from_list(hf_data)

def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        text_target=examples["target_text"],
        max_length=256,
        truncation=True,
        padding="max_length"
    )

    labels = [
        [-100 if token == tokenizer.pad_token_id else token for token in label]
        for label in model_inputs["labels"]
    ]

    return {
        "input_ids": model_inputs["input_ids"],
        "attention_mask": model_inputs["attention_mask"],
        "labels": labels
    }


tokenized_dataset = hf_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["input_text", "target_text"]
)



Map:   0%|          | 0/602 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 602
})

In [ ]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_val_dataset = split_dataset["train"].train_test_split(test_size=0.125)

train_dataset = train_val_dataset["train"]
val_dataset = train_val_dataset["test"]
test_dataset = split_dataset["test"]


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

import sys

model.generation_config.early_stopping = True
model.generation_config.num_beams = 4
model.generation_config.no_repeat_ngram_size = 3
model.generation_config.forced_bos_token_id = tokenizer.bos_token_id

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./pfas_bart_finetuned",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    save_strategy="epoch",
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=3e-5,
    warmup_steps=500,
    weight_decay=0.01,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)

trainer.train()

model.save_pretrained("/content/drive/MyDrive/umich 25 WN/CSE 692 Project/BART/5_classes/land_large_dis_bart/bart_model")
tokenizer.save_pretrained("/content/drive/MyDrive/umich 25 WN/CSE 692 Project/BART/5_classes/bart_model/land_large_dis_bart/bart_model")

In [ ]:
# Evaluate on the test dataset
test_results = trainer.evaluate(eval_dataset=test_dataset)

print(test_results)
# Compute and print perplexity
import math
print(f">>> Test Perplexity: {math.exp(test_results['eval_loss']):.2f}")


{'eval_loss': 0.27900728583335876, 'eval_runtime': 8.8112, 'eval_samples_per_second': 13.732, 'eval_steps_per_second': 1.816, 'epoch': 8.0}
>>> Test Perplexity: 1.32


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_path = "/content/drive/MyDrive/umich 25 WN/CSE 692 Project/BART/5_classes/land_large_dis_bart/bart_model"
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_path)
model.eval().to("cuda")  # or "cpu" if no GPU


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50266, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50266, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:


from transformers import DataCollatorForSeq2Seq
from torch.utils.data import DataLoader


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)



In [ ]:
import torch
predictions = []
references = []

for batch in test_loader:
    input_ids = batch['input_ids'].to("cuda")
    attention_mask = batch['attention_mask'].to("cuda")
    labels = batch['labels']

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)

    decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Fix starts here: replace -100 with tokenizer.pad_token_id, then decode
    labels[labels == -100] = tokenizer.pad_token_id
    decoded_refs = tokenizer.batch_decode(labels, skip_special_tokens=True)

    predictions.extend(decoded_preds)
    references.extend(decoded_refs)


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

results = rouge.compute(predictions=predictions, references=references)
print(results)


{'rouge1': np.float64(0.9161731200304184), 'rouge2': np.float64(0.8798804457737472), 'rougeL': np.float64(0.9164996104084187), 'rougeLsum': np.float64(0.9161619091460149)}


In [ ]:
import evaluate

# Load BLEU metric
bleu = evaluate.load("bleu")


formatted_preds = predictions
formatted_refs = [[ref] for ref in references]  # single reference per prediction

# Compute BLEU
bleu_result = bleu.compute(predictions=formatted_preds, references=formatted_refs)
print(f"BLEU score: {bleu_result['bleu']:.4f}")


BLEU score: 0.8731


In [ ]:
meteor = evaluate.load("meteor")
meteor_result = meteor.compute(predictions=predictions, references=references)
print(meteor_result)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


{'meteor': np.float64(0.9293009033669177)}


In [ ]:
from bert_score import score
P, R, F1 = score(predictions, references, lang="en", verbose=True)
print(f"BERTScore - P: {P.mean().item():.4f}, R: {R.mean().item():.4f}, F1: {F1.mean().item():.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 1.12 seconds, 107.60 sentences/sec
BERTScore - P: 0.9855, R: 0.9842, F1: 0.9848
